In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
train = pd.read_csv(r'D:\tianchi\tuijian\Feature-engineering_and_Parameter_Tuning_XGBoost\fresh_comp_offline\tianchi_fresh_comp_train_user.csv')

1、感受数据：通过各种统计角度进行分析
2、数据清洗：
3、特征提取：
4、样本采样+归一化：
5、模型训练：
6、结果预测：

In [2]:
#train.describe()

In [3]:
#train.head(5)

In [4]:
#print(train.isnull().any()) #user_id、item_id、behavior_type、item_category、time中均没有空数据，只有 user_geohash中有缺失数据
#这意味着数据清洗的工作占时不用做。

In [5]:
print('total:',train.iloc[:,0].size)
print('user_id:',train.groupby('user_id').size().size)
print('item_id:',train.groupby('item_id').size().size)
print('category_id:',train.groupby('item_category').size().size)

total: 23291027
user_id: 20000
item_id: 4758484
category_id: 9557


最后构造出来的数据为：
user_id, goods_id，[行为特征列表]，buy_or_not

part_1 - train: 11.22~11.27 -> 11.28;
part_2 - train: 11.29~12.04 -> 12.05;
part_3 - train: 12.06~12.11 -> 12.12;
part_4 - pred:  12.13~12.18 (-> 12.19);

In [6]:
#part1
#U用户在目标日期前6天的浏览(1)、收藏(2)、加购物车(3)、购买(4)的行为次数，总行为次数。
df_part_1_u_b = train[(train.time >= '2014-11-22 00') & (train.time <= '2014-11-27 23')]
df_part_1_u_b['cumcount'] = df_part_1_u_b.groupby(['user_id','behavior_type']).cumcount()
df_part_1_u_b_count_in_6 = df_part_1_u_b.drop_duplicates(['user_id','behavior_type'], 'last')[['user_id','behavior_type','cumcount']]
df_part_1_u_b_count_in_6 = pd.get_dummies(df_part_1_u_b_count_in_6['behavior_type']).join(df_part_1_u_b_count_in_6[['user_id','cumcount']])
df_part_1_u_b_count_in_6.rename(columns = {1:'behavior_type_1',
                                           2:'behavior_type_2',
                                           3:'behavior_type_3',
                                           4:'behavior_type_4'}, inplace=True)
df_part_1_u_b_count_in_6['u_b1_count_in_6'] = df_part_1_u_b_count_in_6['behavior_type_1'] * (df_part_1_u_b_count_in_6['cumcount']+1)
df_part_1_u_b_count_in_6['u_b2_count_in_6'] = df_part_1_u_b_count_in_6['behavior_type_2'] * (df_part_1_u_b_count_in_6['cumcount']+1)
df_part_1_u_b_count_in_6['u_b3_count_in_6'] = df_part_1_u_b_count_in_6['behavior_type_3'] * (df_part_1_u_b_count_in_6['cumcount']+1)
df_part_1_u_b_count_in_6['u_b4_count_in_6'] = df_part_1_u_b_count_in_6['behavior_type_4'] * (df_part_1_u_b_count_in_6['cumcount']+1)
##print("1")
##print(df_part_1_u_b_count_in_6.head(5))
#这步没做什么，实质是进行了列的选取
df_part_1_u_b_count_in_6 = df_part_1_u_b_count_in_6.groupby('user_id').agg({'u_b1_count_in_6': np.sum,
                                                                            'u_b2_count_in_6': np.sum, 
                                                                            'u_b3_count_in_6': np.sum, 
                                                                            'u_b4_count_in_6': np.sum})
##print("2")
##print(df_part_1_u_b_count_in_6.head(5))
df_part_1_u_b_count_in_6.reset_index(inplace = True)
df_part_1_u_b_count_in_6['u_b_count_in_6'] = df_part_1_u_b_count_in_6[['u_b1_count_in_6',
                                                                       'u_b2_count_in_6',
                                                                       'u_b3_count_in_6',
                                                                       'u_b4_count_in_6']].apply(lambda x: x.sum(), axis = 1)
#--------------------------------------------------------------------------------------------------------
#I商品在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_i_b = df_part_1_u_b   
df_part_1_i_b['i_cumcount'] = df_part_1_i_b.groupby(['item_id','behavior_type']).cumcount()
df_part_1_i_b_count_in_6 = df_part_1_i_b.drop_duplicates(['item_id','behavior_type'], 'last')[['item_id','behavior_type','i_cumcount']]
df_part_1_i_b_count_in_6 = pd.get_dummies(df_part_1_i_b_count_in_6['behavior_type']).join(df_part_1_i_b_count_in_6[['item_id','i_cumcount']])
df_part_1_i_b_count_in_6.rename(columns = {1:'i_behavior_type_1',
                                           2:'i_behavior_type_2',
                                           3:'i_behavior_type_3',
                                           4:'i_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_i_b_count_in_6['i_b1_count_in_6'] = df_part_1_i_b_count_in_6['i_behavior_type_1'] * (df_part_1_i_b_count_in_6['i_cumcount']+1)
df_part_1_i_b_count_in_6['i_b2_count_in_6'] = df_part_1_i_b_count_in_6['i_behavior_type_2'] * (df_part_1_i_b_count_in_6['i_cumcount']+1)
df_part_1_i_b_count_in_6['i_b3_count_in_6'] = df_part_1_i_b_count_in_6['i_behavior_type_3'] * (df_part_1_i_b_count_in_6['i_cumcount']+1)
df_part_1_i_b_count_in_6['i_b4_count_in_6'] = df_part_1_i_b_count_in_6['i_behavior_type_4'] * (df_part_1_i_b_count_in_6['i_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_i_b_count_in_6 = df_part_1_i_b_count_in_6.groupby('item_id').agg({'i_b1_count_in_6': np.sum,
                                                                            'i_b2_count_in_6': np.sum, 
                                                                            'i_b3_count_in_6': np.sum, 
                                                                            'i_b4_count_in_6': np.sum})
df_part_1_i_b_count_in_6.reset_index(inplace = True)
##总行为次数
df_part_1_i_b_count_in_6['i_b_count_in_6'] = df_part_1_i_b_count_in_6[['i_b1_count_in_6',
                                                                       'i_b2_count_in_6',
                                                                       'i_b3_count_in_6',
                                                                       'i_b4_count_in_6']].apply(lambda x: x.sum(), axis = 1)
##商品点击购买转化率
df_part_1_i_b_count_in_6['i_b4_rate_in_6'] = df_part_1_i_b_count_in_6['i_b4_count_in_6'] / df_part_1_i_b_count_in_6['i_b_count_in_6']
##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
item_user_group_size = pd.DataFrame(df_part_1_i_b.groupby(['item_id','user_id']).size())
item_user_num = pd.DataFrame(item_user_group_size.groupby(['item_id']).size())
item_user_num.rename(columns = {0:'i_user_num'},inplace=True)
df_part_1_i_b_count_in_6=df_part_1_i_b_count_in_6.set_index('item_id')
df_part_1_i_b_count_in_6 = df_part_1_i_b_count_in_6.join(item_user_num)
#-----------------------------------------------------------------------------------------------------------
#C类目在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_c_b = df_part_1_u_b
df_part_1_c_b['c_cumcount'] = df_part_1_c_b.groupby(['item_category','behavior_type']).cumcount()
df_part_1_c_b_count_in_6 = df_part_1_c_b.drop_duplicates(['item_category','behavior_type'], 'last')[['item_category','behavior_type','c_cumcount']]
df_part_1_c_b_count_in_6 = pd.get_dummies(df_part_1_c_b_count_in_6['behavior_type']).join(df_part_1_c_b_count_in_6[['item_category','c_cumcount']])
df_part_1_c_b_count_in_6.rename(columns = {1:'c_behavior_type_1',
                                           2:'c_behavior_type_2',
                                           3:'c_behavior_type_3',
                                           4:'c_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_c_b_count_in_6['c_b1_count_in_6'] = df_part_1_c_b_count_in_6['c_behavior_type_1'] * (df_part_1_c_b_count_in_6['c_cumcount']+1)
df_part_1_c_b_count_in_6['c_b2_count_in_6'] = df_part_1_c_b_count_in_6['c_behavior_type_2'] * (df_part_1_c_b_count_in_6['c_cumcount']+1)
df_part_1_c_b_count_in_6['c_b3_count_in_6'] = df_part_1_c_b_count_in_6['c_behavior_type_3'] * (df_part_1_c_b_count_in_6['c_cumcount']+1)
df_part_1_c_b_count_in_6['c_b4_count_in_6'] = df_part_1_c_b_count_in_6['c_behavior_type_4'] * (df_part_1_c_b_count_in_6['c_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_c_b_count_in_6 = df_part_1_c_b_count_in_6.groupby('item_category').agg({'c_b1_count_in_6': np.sum,
                                                                                  'c_b2_count_in_6': np.sum, 
                                                                                  'c_b3_count_in_6': np.sum, 
                                                                                  'c_b4_count_in_6': np.sum})
df_part_1_c_b_count_in_6.reset_index(inplace = True)
##总行为次数
df_part_1_c_b_count_in_6['c_b_count_in_6'] = df_part_1_c_b_count_in_6[['c_b1_count_in_6',
                                                                       'c_b2_count_in_6',
                                                                       'c_b3_count_in_6',
                                                                       'c_b4_count_in_6']].apply(lambda x: x.sum(), axis = 1)
##商品点击购买转化率
df_part_1_c_b_count_in_6['c_b4_rate_in_6'] = df_part_1_c_b_count_in_6['c_b4_count_in_6'] / df_part_1_c_b_count_in_6['c_b_count_in_6']

##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
category_user_group_size = pd.DataFrame(df_part_1_c_b.groupby(['item_category','user_id']).size())
category_user_num = pd.DataFrame(category_user_group_size.groupby(['item_category']).size())
category_user_num.rename(columns = {0:'c_user_num'},inplace=True)
df_part_1_c_b_count_in_6=df_part_1_c_b_count_in_6.set_index(['item_category'])
df_part_1_c_b_count_in_6 = df_part_1_c_b_count_in_6.join(category_user_num)
#---------------------------------------------------------------------------------------
#IC商品在所属类别中的用户人数(排序)、商品在所属类别中的行为总数(排序)、商品在所属类别中的销量排序
df_part_1_ic_b = df_part_1_u_b
##商品在所属类别中的用户人数
item_category_user_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','user_id']).size())
item_category_user_num = pd.DataFrame(item_category_user_size.groupby(['item_id','item_category']).size())
item_category_user_num.rename(columns = {0:'ic_user_num_6'},inplace=True)
#print(item_category_user_num.head(5))
##商品在所属类别中的行为总数
item_category_behavior_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category']).size())
item_category_behavior_size.rename(columns = {0:'ic_behavior_num_6'},inplace=True)
#print('1:',item_category_behavior_size.head(5))
##商品在所属类别中的销量排序
item_category_behavior_sales_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','behavior_type']).size())
item_category_behavior_sales_size = item_category_behavior_sales_size.reset_index()
#print('2:',item_category_behavior_sales_size.head(5))
df_part_1_ic_b_count_in_6 = item_category_behavior_sales_size[item_category_behavior_sales_size.behavior_type==4]
df_part_1_ic_b_count_in_6.rename(columns = {0:'ic_sales_num_6'},inplace=True)
df_part_1_ic_b_count_in_6.set_index(['item_id','item_category'],inplace=True)
#print('3:',df_part_1_ic_b_count_in_6.head(5))
df_part_1_ic_b_count_in_6 = df_part_1_ic_b_count_in_6.join(item_category_user_num,how='outer')
df_part_1_ic_b_count_in_6 = df_part_1_ic_b_count_in_6.join(item_category_behavior_size,how='outer')
df_part_1_ic_b_count_in_6 = df_part_1_ic_b_count_in_6[['ic_sales_num_6','ic_user_num_6','ic_behavior_num_6']].fillna(0)
#df_part_1_ic_b_count_in_6
#---------------------------------------------------------------------------------------
#UI用户-商品对在考察日前6天的行为总数计数、用户-商品对在考察日前6天的各项行为计数、
##用户-商品对各项行为上一次发生距考察日的时差、用户商品对的行为在用户所有商品中的排序
df_part_1_ui_b = df_part_1_u_b
ui_b_count_in_6_size = pd.DataFrame(df_part_1_ui_b.groupby(['item_id','user_id','behavior_type']).size())
ui_b_count_in_6_size = ui_b_count_in_6_size.reset_index()
ui_b_count_in_6_size.rename(columns={0:'ui_cumcount'},inplace=True)
ui_b_count_in_6_num = pd.get_dummies(ui_b_count_in_6_size['behavior_type']).join(ui_b_count_in_6_size[['user_id','item_id','ui_cumcount']])
ui_b_count_in_6_num.rename(columns={1:'ui_b1_count_in_6',
                                    2:'ui_b2_count_in_6',
                                    3:'ui_b3_count_in_6',
                                    4:'ui_b4_count_in_6'},inplace=True)
##用户-商品对在考察日前n天的各项行为计数
ui_b_count_in_6_num['ui_b1_count_in_6'] = ui_b_count_in_6_num['ui_b1_count_in_6'] * ui_b_count_in_6_num['ui_cumcount']
ui_b_count_in_6_num['ui_b2_count_in_6'] = ui_b_count_in_6_num['ui_b2_count_in_6'] * ui_b_count_in_6_num['ui_cumcount']
ui_b_count_in_6_num['ui_b3_count_in_6'] = ui_b_count_in_6_num['ui_b3_count_in_6'] * ui_b_count_in_6_num['ui_cumcount']
ui_b_count_in_6_num['ui_b4_count_in_6'] = ui_b_count_in_6_num['ui_b4_count_in_6'] * ui_b_count_in_6_num['ui_cumcount']
##用户-商品对在考察日前n天的行为总数计数
ui_b_count_in_6_num['ui_b_count_in_6']= ui_b_count_in_6_num['ui_b1_count_in_6'] + ui_b_count_in_6_num['ui_b2_count_in_6'] + ui_b_count_in_6_num['ui_b3_count_in_6'] + ui_b_count_in_6_num['ui_b4_count_in_6']
ui_b_count_in_6_num = ui_b_count_in_6_num.groupby(['item_id','user_id']).sum()
##用户-商品对各项行为上一次发生距考察日的时差
##用户-商品对的行为在本用户所有用户-商品对中的排序
ui_b_count_in_6_num = ui_b_count_in_6_num.sort_values(by='ui_b_count_in_6',ascending=True)
temp = pd.DataFrame(ui_b_count_in_6_num.groupby(['user_id']).cumcount())
ui_b_count_in_6_num = temp.join(ui_b_count_in_6_num)
ui_b_count_in_6_num.rename(columns={0:'ui_sort_6'},inplace=True)
ui_b_count_in_6_num['ui_sort_6'] = ui_b_count_in_6_num['ui_sort_6']+1
#----------------------------------------------------------------------------------------------
"""
#UC用户-类别对在考察日前n天的行为总数计数、用户-类别对在考察日前n天的各项行为计数
#用户-类别对各项行为上一次发生距考察日的时差、用户-类别对的行为在用户所有商品中的排序
df_part_1_uc_b = df_part_1_u_b
uc_b_count_in_6_size = pd.DataFrame(df_part_1_uc_b.groupby(['user_id','item_category','behavior_type']).size())
uc_b_count_in_6_size = uc_b_count_in_6_size.reset_index()
uc_b_count_in_6_size.rename(columns={0:'uc_cumcount'},inplace=True)
uc_b_count_in_6_num = pd.get_dummies(uc_b_count_in_6_size['behavior_type']).join(uc_b_count_in_6_size[['user_id','item_category','uc_cumcount']])
uc_b_count_in_6_num.rename(columns={1:'uc_b1_count_in_6',
                                    2:'uc_b2_count_in_6',
                                    3:'uc_b3_count_in_6',
                                    4:'uc_b4_count_in_6'},inplace=True)
#用户-类别对在考察日前n天的各项行为计数
uc_b_count_in_6_num['uc_b1_count_in_6'] = uc_b_count_in_6_num['uc_b1_count_in_6'] * uc_b_count_in_6_num['uc_cumcount']
uc_b_count_in_6_num['uc_b2_count_in_6'] = uc_b_count_in_6_num['uc_b2_count_in_6'] * uc_b_count_in_6_num['uc_cumcount']
uc_b_count_in_6_num['uc_b3_count_in_6'] = uc_b_count_in_6_num['uc_b3_count_in_6'] * uc_b_count_in_6_num['uc_cumcount']
uc_b_count_in_6_num['uc_b4_count_in_6'] = uc_b_count_in_6_num['uc_b4_count_in_6'] * uc_b_count_in_6_num['uc_cumcount']
#用户-类别对在考察日前n天的行为总数计数
uc_b_count_in_6_num['uc_b_count_in_6']= uc_b_count_in_6_num['uc_b1_count_in_6'] + uc_b_count_in_6_num['uc_b2_count_in_6'] + uc_b_count_in_6_num['uc_b3_count_in_6'] + uc_b_count_in_6_num['uc_b4_count_in_6']
#用户-类别对各项行为上一次发生距考察日的时差
#用户-类别对的行为在用户所有商品中的排序
uc_b_count_in_6_num = uc_b_count_in_6_num.sort_values(by='uc_b_count_in_6',ascending=True)
temp = pd.DataFrame(uc_b_count_in_6_num.groupby(['user_id']).cumcount())
uc_b_count_in_6_num = temp.join(uc_b_count_in_6_num)
uc_b_count_in_6_num.rename(columns={0:'uc_sort'},inplace=True)
uc_b_count_in_6_num['uc_sort'] = uc_b_count_in_6_num['uc_sort']+1
"""
#特征工程暂时到此为止
print('ui')
print(ui_b_count_in_6_num.head(5))
#print('uc')
#print(uc_b_count_in_6_num.head(5))
print('ic')
print(item_category_behavior_sales_size.head(5))
print('u')
print(df_part_1_u_b_count_in_6.head(5))
print('i')
print(df_part_1_i_b_count_in_6.head(5))
print('c')
print(df_part_1_c_b_count_in_6.head(5))
print('train data')
#--------------------
print('U:',df_part_1_u_b_count_in_6.iloc[:,0].size)
print('I:',df_part_1_i_b_count_in_6.iloc[:,0].size)
print('C:',df_part_1_c_b_count_in_6.iloc[:,0].size)
print('UI:',ui_b_count_in_6_num.iloc[:,0].size)
#print('UC:',uc_b_count_in_6_num.iloc[:,0].size)
print('IC:',item_category_behavior_sales_size.iloc[:,0].size)
#---------------------
ui_b_count_in_6_num.reset_index(inplace=True)
ui_b_count_in_6_num.set_index(keys=['item_id'], inplace=True)
ui_b_count_in_6_num.sort_index(inplace=True)
df_part_1_ic_b_count_in_6.reset_index(inplace=True)
df_part_1_ic_b_count_in_6.set_index('item_id',inplace=True)
df_part_1_ic_b_count_in_6.sort_index(inplace=True)
df_part_1_train_data_6 = ui_b_count_in_6_num.join(df_part_1_ic_b_count_in_6,how='outer')
df_part_1_train_data_6 = df_part_1_train_data_6.join(df_part_1_i_b_count_in_6)
#print('rows:',df_part_1_train_data_6.iloc[:,0].size)
df_part_1_train_data_6.reset_index(inplace=True)
df_part_1_train_data_6.set_index(keys=['item_category'],inplace=True)
df_part_1_train_data_6 = df_part_1_train_data_6.join(df_part_1_c_b_count_in_6,how='outer')
#print('rows:',df_part_1_train_data_6.iloc[:,0].size)
df_part_1_train_data_6.reset_index(inplace=True)
df_part_1_train_data_6.set_index(keys=['user_id'],inplace=True)
df_part_1_u_b_count_in_6.set_index(keys=['user_id'],inplace=True)
df_part_1_train_data_6 = df_part_1_train_data_6.join(df_part_1_u_b_count_in_6,how='outer')
#print('rows:',df_part_1_train_data_6.iloc[:,0].size)
#df_part_1_train_data_6.head(5)
#df_part_1_train_data_6.to_csv('df_part_1_train_data_6.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

ui
                     ui_sort_6  ui_b1_count_in_6  ui_b2_count_in_6  \
item_id   user_id                                                    
291871075 61738211           1                 1                 0   
86820228  116150090          1                 1                 0   
218032013 10531113           1                 1                 0   
275843440 134768454          1                 1                 0   
155115845 117293267          1                 1                 0   

                     ui_b3_count_in_6  ui_b4_count_in_6  ui_cumcount  \
item_id   user_id                                                      
291871075 61738211                  0                 0            1   
86820228  116150090                 0                 0            1   
218032013 10531113                  0                 0            1   
275843440 134768454                 0                 0            1   
155115845 117293267                 0                 0            1   

 

In [7]:
#===================================================================================================================
#3天的特征数据的统计
#===================================================================================================================
#U用户在目标日期前3天的浏览(1)、收藏(2)、加购物车(3)、购买(4)的行为次数，总行为数。
df_part_1_u_b = train[(train.time >= '2014-11-25 00') & (train.time <= '2014-11-27 23')]
df_part_1_u_b['cumcount'] = df_part_1_u_b.groupby(['user_id','behavior_type']).cumcount()
df_part_1_u_b_count_in_3 = df_part_1_u_b.drop_duplicates(['user_id','behavior_type'], 'last')[['user_id','behavior_type','cumcount']]

df_part_1_u_b_count_in_3 = pd.get_dummies(df_part_1_u_b_count_in_3['behavior_type']).join(df_part_1_u_b_count_in_3[['user_id','cumcount']])
df_part_1_u_b_count_in_3.rename(columns = {1:'behavior_type_1',
                                           2:'behavior_type_2',
                                           3:'behavior_type_3',
                                           4:'behavior_type_4'}, inplace=True)
df_part_1_u_b_count_in_3['u_b1_count_in_3'] = df_part_1_u_b_count_in_3['behavior_type_1'] * (df_part_1_u_b_count_in_3['cumcount']+1)
df_part_1_u_b_count_in_3['u_b2_count_in_3'] = df_part_1_u_b_count_in_3['behavior_type_2'] * (df_part_1_u_b_count_in_3['cumcount']+1)
df_part_1_u_b_count_in_3['u_b3_count_in_3'] = df_part_1_u_b_count_in_3['behavior_type_3'] * (df_part_1_u_b_count_in_3['cumcount']+1)
df_part_1_u_b_count_in_3['u_b4_count_in_3'] = df_part_1_u_b_count_in_3['behavior_type_4'] * (df_part_1_u_b_count_in_3['cumcount']+1)
df_part_1_u_b_count_in_3 = df_part_1_u_b_count_in_3.groupby('user_id').agg({'u_b1_count_in_3': np.sum,
                                                                            'u_b2_count_in_3': np.sum, 
                                                                            'u_b3_count_in_3': np.sum, 
                                                                            'u_b4_count_in_3': np.sum})
df_part_1_u_b_count_in_3.reset_index(inplace = True)
df_part_1_u_b_count_in_3['u_b_count_in_3'] = df_part_1_u_b_count_in_3[['u_b1_count_in_3',
                                                                       'u_b2_count_in_3',
                                                                       'u_b3_count_in_3',
                                                                       'u_b4_count_in_3']].apply(lambda x: x.sum(), axis = 1)
#--------------------------------------------------------------------------------------------------------
#I商品在目标日前3天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_i_b = df_part_1_u_b
df_part_1_i_b['i_cumcount'] = df_part_1_i_b.groupby(['item_id','behavior_type']).cumcount()
df_part_1_i_b_count_in_3 = df_part_1_i_b.drop_duplicates(['item_id','behavior_type'], 'last')[['item_id','behavior_type','i_cumcount']]
df_part_1_i_b_count_in_3 = pd.get_dummies(df_part_1_i_b_count_in_3['behavior_type']).join(df_part_1_i_b_count_in_3[['item_id','i_cumcount']])
df_part_1_i_b_count_in_3.rename(columns = {1:'i_behavior_type_1',
                                           2:'i_behavior_type_2',
                                           3:'i_behavior_type_3',
                                           4:'i_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_i_b_count_in_3['i_b1_count_in_3'] = df_part_1_i_b_count_in_3['i_behavior_type_1'] * (df_part_1_i_b_count_in_3['i_cumcount']+1)
df_part_1_i_b_count_in_3['i_b2_count_in_3'] = df_part_1_i_b_count_in_3['i_behavior_type_2'] * (df_part_1_i_b_count_in_3['i_cumcount']+1)
df_part_1_i_b_count_in_3['i_b3_count_in_3'] = df_part_1_i_b_count_in_3['i_behavior_type_3'] * (df_part_1_i_b_count_in_3['i_cumcount']+1)
df_part_1_i_b_count_in_3['i_b4_count_in_3'] = df_part_1_i_b_count_in_3['i_behavior_type_4'] * (df_part_1_i_b_count_in_3['i_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_i_b_count_in_3 = df_part_1_i_b_count_in_3.groupby('item_id').agg({'i_b1_count_in_3': np.sum,
                                                                            'i_b2_count_in_3': np.sum, 
                                                                            'i_b3_count_in_3': np.sum, 
                                                                            'i_b4_count_in_3': np.sum})
df_part_1_i_b_count_in_3.reset_index(inplace = True)
##总行为次数
df_part_1_i_b_count_in_3['i_b_count_in_3'] = df_part_1_i_b_count_in_3[['i_b1_count_in_3',
                                                                       'i_b2_count_in_3',
                                                                       'i_b3_count_in_3',
                                                                       'i_b4_count_in_3']].apply(lambda x: x.sum(), axis = 1)
##商品点击购买转化率
df_part_1_i_b_count_in_3['i_b4_rate_in_3'] = df_part_1_i_b_count_in_3['i_b4_count_in_3'] / df_part_1_i_b_count_in_3['i_b_count_in_3']
##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
item_user_group_size = pd.DataFrame(df_part_1_i_b.groupby(['item_id','user_id']).size())
item_user_num = pd.DataFrame(item_user_group_size.groupby(['item_id']).size())
item_user_num.rename(columns = {0:'i_user_num'},inplace=True)
df_part_1_i_b_count_in_3=df_part_1_i_b_count_in_3.set_index(['item_id'])
df_part_1_i_b_count_in_3 = df_part_1_i_b_count_in_3.join(item_user_num)
#-----------------------------------------------------------------------------------------------------------
#C类目在目标日前3天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_c_b = df_part_1_u_b
df_part_1_c_b['c_cumcount'] = df_part_1_c_b.groupby(['item_category','behavior_type']).cumcount()
df_part_1_c_b_count_in_3 = df_part_1_c_b.drop_duplicates(['item_category','behavior_type'], 'last')[['item_category','behavior_type','c_cumcount']]
df_part_1_c_b_count_in_3 = pd.get_dummies(df_part_1_c_b_count_in_3['behavior_type']).join(df_part_1_c_b_count_in_3[['item_category','c_cumcount']])
df_part_1_c_b_count_in_3.rename(columns = {1:'c_behavior_type_1',
                                           2:'c_behavior_type_2',
                                           3:'c_behavior_type_3',
                                           4:'c_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_c_b_count_in_3['c_b1_count_in_3'] = df_part_1_c_b_count_in_3['c_behavior_type_1'] * (df_part_1_c_b_count_in_3['c_cumcount']+1)
df_part_1_c_b_count_in_3['c_b2_count_in_3'] = df_part_1_c_b_count_in_3['c_behavior_type_2'] * (df_part_1_c_b_count_in_3['c_cumcount']+1)
df_part_1_c_b_count_in_3['c_b3_count_in_3'] = df_part_1_c_b_count_in_3['c_behavior_type_3'] * (df_part_1_c_b_count_in_3['c_cumcount']+1)
df_part_1_c_b_count_in_3['c_b4_count_in_3'] = df_part_1_c_b_count_in_3['c_behavior_type_4'] * (df_part_1_c_b_count_in_3['c_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_c_b_count_in_3 = df_part_1_c_b_count_in_3.groupby('item_category').agg({'c_b1_count_in_3': np.sum,
                                                                                  'c_b2_count_in_3': np.sum, 
                                                                                  'c_b3_count_in_3': np.sum, 
                                                                                  'c_b4_count_in_3': np.sum})
df_part_1_c_b_count_in_3.reset_index(inplace = True)
##总行为次数
df_part_1_c_b_count_in_3['c_b_count_in_3'] = df_part_1_c_b_count_in_3[['c_b1_count_in_3',
                                                                       'c_b2_count_in_3',
                                                                       'c_b3_count_in_3',
                                                                       'c_b4_count_in_3']].apply(lambda x: x.sum(), axis = 1)
##类目点击购买转化率
df_part_1_c_b_count_in_6['c_b4_rate_in_3'] = df_part_1_c_b_count_in_3['c_b4_count_in_3'] / df_part_1_c_b_count_in_3['c_b_count_in_3']

##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
category_user_group_size = pd.DataFrame(df_part_1_c_b.groupby(['item_category','user_id']).size())
category_user_num = pd.DataFrame(category_user_group_size.groupby(['item_category']).size())
category_user_num.rename(columns = {0:'c_user_num'},inplace=True)
df_part_1_c_b_count_in_3=df_part_1_c_b_count_in_3.set_index(['item_category'])
df_part_1_c_b_count_in_3 = df_part_1_c_b_count_in_3.join(category_user_num)
#---------------------------------------------------------------------------------------
#IC商品在所属类别中的用户人数(排序)、商品在所属类别中的行为总数(排序)、商品在所属类别中的销量排序
df_part_1_ic_b = df_part_1_u_b
##商品在所属类别中的用户人数
item_category_user_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','user_id']).size())
item_category_user_num = pd.DataFrame(item_category_user_size.groupby(['item_id','item_category']).size())
item_category_user_num.rename(columns = {0:'ic_user_num_3'},inplace=True)
#print(item_category_user_num.head(5))
##商品在所属类别中的行为总数
item_category_behavior_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category']).size())
item_category_behavior_size.rename(columns = {0:'ic_behavior_num_3'},inplace=True)
#print('1:',item_category_behavior_size.head(5))
##商品在所属类别中的销量排序
item_category_behavior_sales_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','behavior_type']).size())
item_category_behavior_sales_size = item_category_behavior_sales_size.reset_index()
#print('2:',item_category_behavior_sales_size.head(5))
df_part_1_ic_b_count_in_3 = item_category_behavior_sales_size[item_category_behavior_sales_size.behavior_type==4]
df_part_1_ic_b_count_in_3.rename(columns = {0:'ic_sales_num_3'},inplace=True)
df_part_1_ic_b_count_in_3.set_index(['item_id','item_category'],inplace=True)
#print('3:',df_part_1_ic_b_count_in_6.head(5))
df_part_1_ic_b_count_in_3 = df_part_1_ic_b_count_in_3.join(item_category_user_num,how='outer')
df_part_1_ic_b_count_in_3 = df_part_1_ic_b_count_in_3.join(item_category_behavior_size,how='outer')
df_part_1_ic_b_count_in_3 = df_part_1_ic_b_count_in_3[['ic_sales_num_3','ic_user_num_3','ic_behavior_num_3']].fillna(0)
#df_part_1_ic_b_count_in_3
#---------------------------------------------------------------------------------------
#UI用户-商品对在考察日前n天的行为总数计数、用户-商品对在考察日前n天的各项行为计数、
##用户-商品对各项行为上一次发生距考察日的时差、用户商品对的行为在用户所有商品中的排序
df_part_1_ui_b = df_part_1_u_b
ui_b_count_in_3_size = pd.DataFrame(df_part_1_ui_b.groupby(['item_id','user_id','behavior_type']).size())
ui_b_count_in_3_size = ui_b_count_in_3_size.reset_index()
ui_b_count_in_3_size.rename(columns={0:'ui_cumcount'},inplace=True)
ui_b_count_in_3_num = pd.get_dummies(ui_b_count_in_3_size['behavior_type']).join(ui_b_count_in_3_size[['user_id','item_id','ui_cumcount']])
ui_b_count_in_3_num.rename(columns={1:'ui_b1_count_in_3',
                                    2:'ui_b2_count_in_3',
                                    3:'ui_b3_count_in_3',
                                    4:'ui_b4_count_in_3'},inplace=True)
##用户-商品对在考察日前n天的各项行为计数
ui_b_count_in_3_num['ui_b1_count_in_3'] = ui_b_count_in_3_num['ui_b1_count_in_3'] * ui_b_count_in_3_num['ui_cumcount']
ui_b_count_in_3_num['ui_b2_count_in_3'] = ui_b_count_in_3_num['ui_b2_count_in_3'] * ui_b_count_in_3_num['ui_cumcount']
ui_b_count_in_3_num['ui_b3_count_in_3'] = ui_b_count_in_3_num['ui_b3_count_in_3'] * ui_b_count_in_3_num['ui_cumcount']
ui_b_count_in_3_num['ui_b4_count_in_3'] = ui_b_count_in_3_num['ui_b4_count_in_3'] * ui_b_count_in_3_num['ui_cumcount']
##用户-商品对在考察日前n天的行为总数计数
ui_b_count_in_3_num['ui_b_count_in_3']= ui_b_count_in_3_num['ui_b1_count_in_3'] + ui_b_count_in_3_num['ui_b2_count_in_3'] + ui_b_count_in_3_num['ui_b3_count_in_3'] + ui_b_count_in_3_num['ui_b4_count_in_3']
ui_b_count_in_3_num = ui_b_count_in_3_num.groupby(['item_id','user_id']).sum()
##用户-商品对各项行为上一次发生距考察日的时差
##用户-商品对的行为在本用户所有用户-商品对中的排序
ui_b_count_in_3_num = ui_b_count_in_3_num.sort_values(by='ui_b_count_in_3',ascending=True)
temp = pd.DataFrame(ui_b_count_in_3_num.groupby(['user_id']).cumcount())
ui_b_count_in_3_num = temp.join(ui_b_count_in_3_num)
ui_b_count_in_3_num.rename(columns={0:'ui_sort_3'},inplace=True)
ui_b_count_in_3_num['ui_sort_3'] = ui_b_count_in_3_num['ui_sort_3']+1
#----------------------------------------------------------------------------------------------
"""
#UC用户-类别对在考察日前n天的行为总数计数、用户-类别对在考察日前n天的各项行为计数
#用户-类别对各项行为上一次发生距考察日的时差、用户-类别对的行为在用户所有商品中的排序
df_part_1_uc_b = df_part_1_u_b
uc_b_count_in_3_size = pd.DataFrame(df_part_1_uc_b.groupby(['user_id','item_category','behavior_type']).size())
uc_b_count_in_3_size = uc_b_count_in_3_size.reset_index()
uc_b_count_in_3_size.rename(columns={0:'uc_cumcount'},inplace=True)
uc_b_count_in_3_num = pd.get_dummies(uc_b_count_in_3_size['behavior_type']).join(uc_b_count_in_3_size[['user_id','item_category','uc_cumcount']])
uc_b_count_in_3_num.rename(columns={1:'uc_b1_count_in_3',
                                    2:'uc_b2_count_in_3',
                                    3:'uc_b3_count_in_3',
                                    4:'uc_b4_count_in_3'},inplace=True)
#用户-类别对在考察日前n天的各项行为计数
uc_b_count_in_3_num['uc_b1_count_in_3'] = uc_b_count_in_3_num['uc_b1_count_in_3'] * uc_b_count_in_3_num['uc_cumcount']
uc_b_count_in_3_num['uc_b2_count_in_3'] = uc_b_count_in_3_num['uc_b2_count_in_3'] * uc_b_count_in_3_num['uc_cumcount']
uc_b_count_in_3_num['uc_b3_count_in_3'] = uc_b_count_in_3_num['uc_b3_count_in_3'] * uc_b_count_in_3_num['uc_cumcount']
uc_b_count_in_3_num['uc_b4_count_in_3'] = uc_b_count_in_3_num['uc_b4_count_in_3'] * uc_b_count_in_3_num['uc_cumcount']
#用户-类别对在考察日前n天的行为总数计数
uc_b_count_in_3_num['uc_b_count_in_3']= uc_b_count_in_3_num['uc_b1_count_in_3'] + uc_b_count_in_3_num['uc_b2_count_in_3'] + uc_b_count_in_3_num['uc_b3_count_in_3'] + uc_b_count_in_3_num['uc_b4_count_in_3']
#用户-类别对各项行为上一次发生距考察日的时差
#用户-类别对的行为在用户所有商品中的排序
uc_b_count_in_3_num = uc_b_count_in_3_num.sort_values(by='uc_b_count_in_3',ascending=True)
temp = pd.DataFrame(uc_b_count_in_3_num.groupby(['user_id']).cumcount())
uc_b_count_in_3_num = temp.join(uc_b_count_in_3_num,on=['user_id','item_category'])
uc_b_count_in_3_num.rename(columns={0:'uc_sort_3'},inplace=True)
uc_b_count_in_3_num['uc_sort_3'] = uc_b_count_in_3_num['uc_sort_3']+1
"""
#特征工程暂时到此为止
print('ui')
print(ui_b_count_in_3_num.head(5))
#print('uc')
#print(uc_b_count_in_3_num.head(5))
print('ic')
print(item_category_behavior_sales_size.head(5))
print('u')
print(df_part_1_u_b_count_in_3.head(5))
print('i')
print(df_part_1_i_b_count_in_3.head(5))
print('c')
print(df_part_1_c_b_count_in_3.head(5))
print('train data')
#--------------------
print('U:',df_part_1_u_b_count_in_3.iloc[:,0].size)
print('I:',df_part_1_i_b_count_in_3.iloc[:,0].size)
print('C:',df_part_1_c_b_count_in_3.iloc[:,0].size)
print('UI:',ui_b_count_in_3_num.iloc[:,0].size)
#print('UC:',uc_b_count_in_3_num.iloc[:,0].size)
print('IC:',item_category_behavior_sales_size.iloc[:,0].size)
#---------------------
ui_b_count_in_3_num.reset_index(inplace=True)
ui_b_count_in_3_num.set_index(keys=['item_id'], inplace=True)
ui_b_count_in_3_num.sort_index(inplace=True)
df_part_1_ic_b_count_in_3.reset_index(inplace=True)
df_part_1_ic_b_count_in_3.set_index('item_id',inplace=True)
df_part_1_ic_b_count_in_3.sort_index(inplace=True)
df_part_1_train_data_3 = ui_b_count_in_3_num.join(df_part_1_ic_b_count_in_3,how='outer')
df_part_1_train_data_3 = df_part_1_train_data_3.join(df_part_1_i_b_count_in_3)
#print('rows:',df_part_1_train_data_3.iloc[:,0].size)
df_part_1_train_data_3.reset_index(inplace=True)
df_part_1_train_data_3.set_index(keys=['item_category'],inplace=True)
df_part_1_train_data_3 = df_part_1_train_data_3.join(df_part_1_c_b_count_in_3,how='outer')
#print('rows:',df_part_1_train_data_3.iloc[:,0].size)
df_part_1_train_data_3.reset_index(inplace=True)
df_part_1_train_data_3.set_index(keys=['user_id'],inplace=True)
df_part_1_u_b_count_in_3.set_index(keys=['user_id'],inplace=True)
df_part_1_train_data_3 = df_part_1_train_data_3.join(df_part_1_u_b_count_in_3,how='outer')
#print('rows:',df_part_1_train_data_3.iloc[:,0].size)
#df_part_1_train_data_3.head(5)
#df_part_1_train_data_3.to_csv('df_part_1_train_data_3.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

ui
                     ui_sort_3  ui_b1_count_in_3  ui_b2_count_in_3  \
item_id   user_id                                                    
175829719 57477921           1                 1                 0   
175240875 22348842           1                 1                 0   
175237940 53165003           1                 1                 0   
175237712 115575833          1                 1                 0   
175237430 28591349           1                 1                 0   

                     ui_b3_count_in_3  ui_b4_count_in_3  ui_cumcount  \
item_id   user_id                                                      
175829719 57477921                  0                 0            1   
175240875 22348842                  0                 0            1   
175237940 53165003                  0                 0            1   
175237712 115575833                 0                 0            1   
175237430 28591349                  0                 0            1   

 

In [8]:
#################################################################################
#1天的用户行为数据统计
####################################################################################
#U用户在目标日期前1天的1,2,3,4的行为数，总行为数。
df_part_1_u_b = train[(train.time >= '2014-11-27 00') & (train.time <= '2014-11-27 23')]
df_part_1_u_b['cumcount'] = df_part_1_u_b.groupby(['user_id','behavior_type']).cumcount()
df_part_1_u_b_count_in_1 = df_part_1_u_b.drop_duplicates(['user_id','behavior_type'], 'last')[['user_id','behavior_type','cumcount']]
df_part_1_u_b_count_in_1 = pd.get_dummies(df_part_1_u_b_count_in_1['behavior_type']).join(df_part_1_u_b_count_in_1[['user_id','cumcount']])
df_part_1_u_b_count_in_1.rename(columns = {1:'behavior_type_1',
                                           2:'behavior_type_2',
                                           3:'behavior_type_3',
                                           4:'behavior_type_4'}, inplace=True)
df_part_1_u_b_count_in_1['u_b1_count_in_1'] = df_part_1_u_b_count_in_1['behavior_type_1'] * (df_part_1_u_b_count_in_1['cumcount']+1)
df_part_1_u_b_count_in_1['u_b2_count_in_1'] = df_part_1_u_b_count_in_1['behavior_type_2'] * (df_part_1_u_b_count_in_1['cumcount']+1)
df_part_1_u_b_count_in_1['u_b3_count_in_1'] = df_part_1_u_b_count_in_1['behavior_type_3'] * (df_part_1_u_b_count_in_1['cumcount']+1)
df_part_1_u_b_count_in_1['u_b4_count_in_1'] = df_part_1_u_b_count_in_1['behavior_type_4'] * (df_part_1_u_b_count_in_1['cumcount']+1)
df_part_1_u_b_count_in_1 = df_part_1_u_b_count_in_1.groupby('user_id').agg({'u_b1_count_in_1': np.sum,
                                                                            'u_b2_count_in_1': np.sum, 
                                                                            'u_b3_count_in_1': np.sum, 
                                                                            'u_b4_count_in_1': np.sum})
df_part_1_u_b_count_in_1.reset_index(inplace = True)
df_part_1_u_b_count_in_1['u_b_count_in_1'] = df_part_1_u_b_count_in_1[['u_b1_count_in_1',
                                                                       'u_b2_count_in_1',
                                                                       'u_b3_count_in_1',
                                                                       'u_b4_count_in_1']].apply(lambda x: x.sum(), axis = 1)
#--------------------------------------------------------------------------------------------------------
#I商品在目标日前1天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_i_b = df_part_1_u_b
df_part_1_i_b['i_cumcount'] = df_part_1_i_b.groupby(['item_id','behavior_type']).cumcount()
df_part_1_i_b_count_in_1 = df_part_1_i_b.drop_duplicates(['item_id','behavior_type'], 'last')[['item_id','behavior_type','i_cumcount']]
df_part_1_i_b_count_in_1 = pd.get_dummies(df_part_1_i_b_count_in_1['behavior_type']).join(df_part_1_i_b_count_in_1[['item_id','i_cumcount']])
df_part_1_i_b_count_in_1.rename(columns = {1:'i_behavior_type_1',
                                           2:'i_behavior_type_2',
                                           3:'i_behavior_type_3',
                                           4:'i_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_i_b_count_in_1['i_b1_count_in_1'] = df_part_1_i_b_count_in_1['i_behavior_type_1'] * (df_part_1_i_b_count_in_1['i_cumcount']+1)
df_part_1_i_b_count_in_1['i_b2_count_in_1'] = df_part_1_i_b_count_in_1['i_behavior_type_2'] * (df_part_1_i_b_count_in_1['i_cumcount']+1)
df_part_1_i_b_count_in_1['i_b3_count_in_1'] = df_part_1_i_b_count_in_1['i_behavior_type_3'] * (df_part_1_i_b_count_in_1['i_cumcount']+1)
df_part_1_i_b_count_in_1['i_b4_count_in_1'] = df_part_1_i_b_count_in_1['i_behavior_type_4'] * (df_part_1_i_b_count_in_1['i_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_i_b_count_in_1 = df_part_1_i_b_count_in_1.groupby('item_id').agg({'i_b1_count_in_1': np.sum,
                                                                            'i_b2_count_in_1': np.sum, 
                                                                            'i_b3_count_in_1': np.sum, 
                                                                            'i_b4_count_in_1': np.sum})
df_part_1_i_b_count_in_1.reset_index(inplace = True)
##总行为次数
df_part_1_i_b_count_in_1['i_b_count_in_1'] = df_part_1_i_b_count_in_1[['i_b1_count_in_1',
                                                                       'i_b2_count_in_1',
                                                                       'i_b3_count_in_1',
                                                                       'i_b4_count_in_1']].apply(lambda x: x.sum(), axis = 1)
##商品点击购买转化率
df_part_1_i_b_count_in_1['i_b4_rate_in_1'] = df_part_1_i_b_count_in_1['i_b4_count_in_1'] / df_part_1_i_b_count_in_1['i_b_count_in_1']
##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
item_user_group_size = pd.DataFrame(df_part_1_i_b.groupby(['item_id','user_id']).size())
item_user_num = pd.DataFrame(item_user_group_size.groupby(['item_id']).size())
item_user_num.rename(columns = {0:'i_user_num'},inplace=True)
df_part_1_i_b_count_in_1 = df_part_1_i_b_count_in_1.set_index(['item_id'])
df_part_1_i_b_count_in_1 = df_part_1_i_b_count_in_1.join(item_user_num)
#-----------------------------------------------------------------------------------------------------------
#C类目在目标日前1天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
df_part_1_c_b = df_part_1_u_b
df_part_1_c_b['c_cumcount'] = df_part_1_c_b.groupby(['item_category','behavior_type']).cumcount()
df_part_1_c_b_count_in_1 = df_part_1_c_b.drop_duplicates(['item_category','behavior_type'], 'last')[['item_category','behavior_type','c_cumcount']]
df_part_1_c_b_count_in_1 = pd.get_dummies(df_part_1_c_b_count_in_1['behavior_type']).join(df_part_1_c_b_count_in_1[['item_category','c_cumcount']])
df_part_1_c_b_count_in_1.rename(columns = {1:'c_behavior_type_1',
                                           2:'c_behavior_type_2',
                                           3:'c_behavior_type_3',
                                           4:'c_behavior_type_4'}, inplace=True)
##各类行为次数
df_part_1_c_b_count_in_1['c_b1_count_in_1'] = df_part_1_c_b_count_in_1['c_behavior_type_1'] * (df_part_1_c_b_count_in_1['c_cumcount']+1)
df_part_1_c_b_count_in_1['c_b2_count_in_1'] = df_part_1_c_b_count_in_1['c_behavior_type_2'] * (df_part_1_c_b_count_in_1['c_cumcount']+1)
df_part_1_c_b_count_in_1['c_b3_count_in_1'] = df_part_1_c_b_count_in_1['c_behavior_type_3'] * (df_part_1_c_b_count_in_1['c_cumcount']+1)
df_part_1_c_b_count_in_1['c_b4_count_in_1'] = df_part_1_c_b_count_in_1['c_behavior_type_4'] * (df_part_1_c_b_count_in_1['c_cumcount']+1)
##这步其实没做什么，实质是进行了列的选取
df_part_1_c_b_count_in_1 = df_part_1_c_b_count_in_1.groupby('item_category').agg({'c_b1_count_in_1': np.sum,
                                                                                  'c_b2_count_in_1': np.sum, 
                                                                                  'c_b3_count_in_1': np.sum, 
                                                                                  'c_b4_count_in_1': np.sum})
df_part_1_c_b_count_in_1.reset_index(inplace = True)
##总行为次数
df_part_1_c_b_count_in_1['c_b_count_in_1'] = df_part_1_c_b_count_in_1[['c_b1_count_in_1',
                                                                       'c_b2_count_in_1',
                                                                       'c_b3_count_in_1',
                                                                       'c_b4_count_in_1']].apply(lambda x: x.sum(), axis = 1)
##商品点击购买转化率
df_part_1_c_b_count_in_1['c_b4_rate_in_1'] = df_part_1_c_b_count_in_1['c_b4_count_in_1'] / df_part_1_c_b_count_in_1['c_b_count_in_1']

##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
#暂略

##被访问人数
category_user_group_size = pd.DataFrame(df_part_1_c_b.groupby(['item_category','user_id']).size())
category_user_num = pd.DataFrame(category_user_group_size.groupby(['item_category']).size())
category_user_num.rename(columns = {0:'c_user_num'},inplace=True)
df_part_1_c_b_count_in_1 = df_part_1_c_b_count_in_1.set_index(['item_category'])
df_part_1_c_b_count_in_1 = df_part_1_c_b_count_in_1.join(category_user_num)
#---------------------------------------------------------------------------------------
#IC商品在所属类别中的用户人数(排序)、商品在所属类别中的行为总数(排序)、商品在所属类别中的销量排序
df_part_1_ic_b = df_part_1_u_b
##商品在所属类别中的用户人数
item_category_user_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','user_id']).size())
item_category_user_num = pd.DataFrame(item_category_user_size.groupby(['item_id','item_category']).size())
item_category_user_num.rename(columns = {0:'ic_user_num_1'},inplace=True)
#print(item_category_user_num.head(5))
##商品在所属类别中的行为总数
item_category_behavior_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category']).size())
item_category_behavior_size.rename(columns = {0:'ic_behavior_num_1'},inplace=True)
#print('1:',item_category_behavior_size.head(5))
##商品在所属类别中的销量排序
item_category_behavior_sales_size = pd.DataFrame(df_part_1_ic_b.groupby(['item_id','item_category','behavior_type']).size())
item_category_behavior_sales_size = item_category_behavior_sales_size.reset_index()
#print('2:',item_category_behavior_sales_size.head(5))
df_part_1_ic_b_count_in_1 = item_category_behavior_sales_size[item_category_behavior_sales_size.behavior_type==4]
df_part_1_ic_b_count_in_1.rename(columns = {0:'ic_sales_num_1'},inplace=True)
df_part_1_ic_b_count_in_1.set_index(['item_id','item_category'],inplace=True)
#print('3:',df_part_1_ic_b_count_in_6.head(5))
df_part_1_ic_b_count_in_1 = df_part_1_ic_b_count_in_1.join(item_category_user_num,how='outer')
df_part_1_ic_b_count_in_1 = df_part_1_ic_b_count_in_1.join(item_category_behavior_size,how='outer')
df_part_1_ic_b_count_in_1 = df_part_1_ic_b_count_in_1[['ic_sales_num_1','ic_user_num_1','ic_behavior_num_1']].fillna(0)
#df_part_1_ic_b_count_in_1
#---------------------------------------------------------------------------------------
#UI用户-商品对在考察日前n天的行为总数计数、用户-商品对在考察日前n天的各项行为计数、
##用户-商品对各项行为上一次发生距考察日的时差、用户商品对的行为在用户所有商品中的排序
df_part_1_ui_b = df_part_1_u_b
ui_b_count_in_1_size = pd.DataFrame(df_part_1_ui_b.groupby(['item_id','user_id','behavior_type']).size())
ui_b_count_in_1_size = ui_b_count_in_1_size.reset_index()
ui_b_count_in_1_size.rename(columns={0:'ui_cumcount'},inplace=True)
ui_b_count_in_1_num = pd.get_dummies(ui_b_count_in_1_size['behavior_type']).join(ui_b_count_in_1_size[['user_id','item_id','ui_cumcount']])
ui_b_count_in_1_num.rename(columns={1:'ui_b1_count_in_1',
                                    2:'ui_b2_count_in_1',
                                    3:'ui_b3_count_in_1',
                                    4:'ui_b4_count_in_1'},inplace=True)
##用户-商品对在考察日前n天的各项行为计数
ui_b_count_in_1_num['ui_b1_count_in_1'] = ui_b_count_in_1_num['ui_b1_count_in_1'] * ui_b_count_in_1_num['ui_cumcount']
ui_b_count_in_1_num['ui_b2_count_in_1'] = ui_b_count_in_1_num['ui_b2_count_in_1'] * ui_b_count_in_1_num['ui_cumcount']
ui_b_count_in_1_num['ui_b3_count_in_1'] = ui_b_count_in_1_num['ui_b3_count_in_1'] * ui_b_count_in_1_num['ui_cumcount']
ui_b_count_in_1_num['ui_b4_count_in_1'] = ui_b_count_in_1_num['ui_b4_count_in_1'] * ui_b_count_in_1_num['ui_cumcount']
##用户-商品对在考察日前n天的行为总数计数
ui_b_count_in_1_num['ui_b_count_in_1']= ui_b_count_in_1_num['ui_b1_count_in_1'] + ui_b_count_in_1_num['ui_b2_count_in_1'] + ui_b_count_in_1_num['ui_b3_count_in_1'] + ui_b_count_in_1_num['ui_b4_count_in_1']
ui_b_count_in_1_num = ui_b_count_in_1_num.groupby(['item_id','user_id']).sum()
##用户-商品对各项行为上一次发生距考察日的时差
##用户-商品对的行为在本用户所有用户-商品对中的排序
ui_b_count_in_1_num = ui_b_count_in_1_num.sort_values(by='ui_b_count_in_1',ascending=True)
temp = pd.DataFrame(ui_b_count_in_1_num.groupby(['user_id']).cumcount())
ui_b_count_in_1_num = temp.join(ui_b_count_in_1_num)
ui_b_count_in_1_num.rename(columns={0:'ui_sort_1'},inplace=True)
ui_b_count_in_1_num['ui_sort_1'] = ui_b_count_in_1_num['ui_sort_1']+1
#----------------------------------------------------------------------------------------------
"""
#UC用户-类别对在考察日前n天的行为总数计数、用户-类别对在考察日前n天的各项行为计数
#用户-类别对各项行为上一次发生距考察日的时差、用户-类别对的行为在用户所有商品中的排序
df_part_1_uc_b = df_part_1_u_b
uc_b_count_in_1_size = pd.DataFrame(df_part_1_uc_b.groupby(['user_id','item_category','behavior_type']).size())
uc_b_count_in_1_size = uc_b_count_in_1_size.reset_index()
uc_b_count_in_1_size.rename(columns={0:'uc_cumcount'},inplace=True)
uc_b_count_in_1_num = pd.get_dummies(uc_b_count_in_1_size['behavior_type']).join(uc_b_count_in_1_size[['user_id','item_category','uc_cumcount']])
uc_b_count_in_1_num.rename(columns={1:'uc_b1_count_in_1',
                                    2:'uc_b2_count_in_1',
                                    3:'uc_b3_count_in_1',
                                    4:'uc_b4_count_in_1'},inplace=True)
#用户-类别对在考察日前n天的各项行为计数
uc_b_count_in_1_num['uc_b1_count_in_1'] = uc_b_count_in_1_num['uc_b1_count_in_1'] * uc_b_count_in_1_num['uc_cumcount']
uc_b_count_in_1_num['uc_b2_count_in_1'] = uc_b_count_in_1_num['uc_b2_count_in_1'] * uc_b_count_in_1_num['uc_cumcount']
uc_b_count_in_1_num['uc_b3_count_in_1'] = uc_b_count_in_1_num['uc_b3_count_in_1'] * uc_b_count_in_1_num['uc_cumcount']
uc_b_count_in_1_num['uc_b4_count_in_1'] = uc_b_count_in_1_num['uc_b4_count_in_1'] * uc_b_count_in_1_num['uc_cumcount']
#用户-类别对在考察日前n天的行为总数计数
uc_b_count_in_1_num['uc_b_count_in_1']= uc_b_count_in_1_num['uc_b1_count_in_1'] + uc_b_count_in_1_num['uc_b2_count_in_1'] + uc_b_count_in_1_num['uc_b3_count_in_1'] + uc_b_count_in_1_num['uc_b4_count_in_1']
#用户-类别对各项行为上一次发生距考察日的时差
#用户-类别对的行为在用户所有商品中的排序
uc_b_count_in_1_num = uc_b_count_in_1_num.sort_values(by='uc_b_count_in_1',ascending=True)
temp = pd.DataFrame(uc_b_count_in_1_num.groupby(['user_id']).cumcount())
uc_b_count_in_1_num = temp.join(uc_b_count_in_1_num,on=['user_id','item_category'])
uc_b_count_in_1_num.rename(columns={0:'uc_sort_1'},inplace=True)
uc_b_count_in_1_num['uc_sort_1'] = uc_b_count_in_1_num['uc_sort_1']+1
"""
#特征工程暂时到此为止
print('ui')
print(ui_b_count_in_1_num.head(5))
#print('uc')
#print(uc_b_count_in_1_num.head(5))
print('ic')
print(item_category_behavior_sales_size.head(5))
print('u')
print(df_part_1_u_b_count_in_1.head(5))
print('i')
print(df_part_1_i_b_count_in_1.head(5))
print('c')
print(df_part_1_c_b_count_in_1.head(5))
print('train data')
#--------------------
print('U:',df_part_1_u_b_count_in_1.iloc[:,0].size)
print('I:',df_part_1_i_b_count_in_1.iloc[:,0].size)
print('C:',df_part_1_c_b_count_in_1.iloc[:,0].size)
print('UI:',ui_b_count_in_1_num.iloc[:,0].size)
#print('UC:',uc_b_count_in_1_num.iloc[:,0].size)
print('IC:',item_category_behavior_sales_size.iloc[:,0].size)
#---------------------
ui_b_count_in_1_num.reset_index(inplace=True)
ui_b_count_in_1_num.set_index(keys=['item_id'], inplace=True)
ui_b_count_in_1_num.sort_index(inplace=True)
df_part_1_ic_b_count_in_1.reset_index(inplace=True)
df_part_1_ic_b_count_in_1.set_index('item_id',inplace=True)
df_part_1_ic_b_count_in_1.sort_index(inplace=True)
df_part_1_train_data_1 = ui_b_count_in_1_num.join(df_part_1_ic_b_count_in_1,how='outer')
df_part_1_train_data_1 = df_part_1_train_data_1.join(df_part_1_i_b_count_in_1)
#print('rows:',df_part_1_train_data_1.iloc[:,0].size)
df_part_1_train_data_1.reset_index(inplace=True)
df_part_1_train_data_1.set_index(keys=['item_category'],inplace=True)
df_part_1_train_data_1 = df_part_1_train_data_1.join(df_part_1_c_b_count_in_1,how='outer')
#print('rows:',df_part_1_train_data_1.iloc[:,0].size)
df_part_1_train_data_1.reset_index(inplace=True)
df_part_1_train_data_1.set_index(keys=['user_id'],inplace=True)
df_part_1_u_b_count_in_1.set_index(keys=['user_id'],inplace=True)
df_part_1_train_data_1 = df_part_1_train_data_1.join(df_part_1_u_b_count_in_1,how='outer')
#print('rows:',df_part_1_train_data_1.iloc[:,0].size)
#df_part_1_train_data_1.head(5)
#df_part_1_train_data_1.to_csv('df_part_1_train_data_1.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

ui
                     ui_sort_1  ui_b1_count_in_1  ui_b2_count_in_1  \
item_id   user_id                                                    
340997328 62909306           1                 1                 0   
238205548 20167692           1                 1                 0   
238206150 111124222          1                 1                 0   
84878169  114635819          1                 1                 0   
          102516628          1                 1                 0   

                     ui_b3_count_in_1  ui_b4_count_in_1  ui_cumcount  \
item_id   user_id                                                      
340997328 62909306                  0                 0            1   
238205548 20167692                  0                 0            1   
238206150 111124222                 0                 0            1   
84878169  114635819                 0                 0            1   
          102516628                 0                 0            1   

 

In [9]:
###
df_part_1_train_data_6.reset_index(inplace=True)
#df_part_1_train_data_6.set_index(keys=['user_id','item_id'], inplace=True)
df_part_1_train_data_3.reset_index(inplace=True)
#df_part_1_train_data_3.set_index(keys=['user_id','item_id'], inplace=True)
df_part_1_train_data_1.reset_index(inplace=True)
#df_part_1_train_data_1.set_index(keys=['user_id','item_id'], inplace=True)
df_part_1_train_data_ = pd.merge(df_part_1_train_data_6, df_part_1_train_data_3, on = ['user_id','item_id'], how = 'left').fillna(0)
df_part_1_train_data_ = pd.merge(df_part_1_train_data_, df_part_1_train_data_1, on = ['user_id','item_id'], how = 'left').fillna(0)

In [10]:
"""
#按照前六天的行为来记录，来生成是否够来的标记数据
df_part_1_train_data_0 = df_part_1_train_data_[df_part_1_train_data_.u_b4_count_in_6 == 0]
df_part_1_train_data_1 = df_part_1_train_data_[df_part_1_train_data_.u_b4_count_in_6 > 0]
df_part_1_train_data_0['buy'] = 0
df_part_1_train_data_1['buy'] = 1
#print(df_part_1_u_b_count_1.shape)
#df_part_1_u_b_count = df_part_1_u_b_count_0.append(df_part_1_u_b_count_1)
##df_part_1_u_b_count.head(5)
print(df_part_1_train_data_0.shape)
print(df_part_1_train_data_1.shape)
#一个反直觉的数据，六天中有行为的数据中竟然75%都买了！！！
"""

"\n#按照前六天的行为来记录，来生成是否够来的标记数据\ndf_part_1_train_data_0 = df_part_1_train_data_[df_part_1_train_data_.u_b4_count_in_6 == 0]\ndf_part_1_train_data_1 = df_part_1_train_data_[df_part_1_train_data_.u_b4_count_in_6 > 0]\ndf_part_1_train_data_0['buy'] = 0\ndf_part_1_train_data_1['buy'] = 1\n#print(df_part_1_u_b_count_1.shape)\n#df_part_1_u_b_count = df_part_1_u_b_count_0.append(df_part_1_u_b_count_1)\n##df_part_1_u_b_count.head(5)\nprint(df_part_1_train_data_0.shape)\nprint(df_part_1_train_data_1.shape)\n#一个反直觉的数据，六天中有行为的数据中竟然75%都买了！！！\n"

In [12]:
#这里会发现某个用户会对某个商品买两次甚至更多。
df_part_1_result = train[(train.time >= '2014-11-28 00') & (train.time <= '2014-11-28 23')]
print('11月28日的用户行为条数：',df_part_1_result.shape)
df_part_1_result = df_part_1_result[df_part_1_result.behavior_type==4]
df_part_1_result['buy'] = 1
df_part_1_result = df_part_1_result[['user_id','item_id','buy']]
df_part_1_result.set_index(keys=['user_id','item_id'],inplace=True)
#df_part_1_result.head(5)
df_part_1_result.reset_index(inplace=True)
df_part_1_result.drop_duplicates(['user_id','item_id'],'last',True)
print('11月28日购买量：',df_part_1_result.shape)
df_part_1_train_data_temp = pd.merge(df_part_1_train_data_, df_part_1_result, on = ['user_id','item_id'], how = 'left',validate = '1:1').fillna(0)
df_part_1_train_data_1 = df_part_1_train_data_temp[df_part_1_train_data_temp.buy > 0]
df_part_1_train_data_0 = df_part_1_train_data_temp[df_part_1_train_data_temp.buy == 0]
print('part_1 正样例：',df_part_1_train_data_1.shape)
print('当天直接购买量：',(df_part_1_result.shape)[0] - (df_part_1_train_data_1.shape)[0])
print('当天直接购买占比：',((df_part_1_result.shape)[0] - (df_part_1_train_data_1.shape)[0])/(df_part_1_result.shape)[0])
print('part_1 负样例：',df_part_1_train_data_0.shape)
df_part_1_train_data_temp.to_csv('df_part_1_train_data.csv')
print('end')

11月28日的用户行为条数： (658806, 6)
11月28日购买量： (5344, 3)
part_1 正样例： (1445, 92)
part_1 负样例： (1707539, 92)


In [ ]:
dateparse = pd.datetime.strptime('2014-11-24 20', '%Y-%m-%d %H')
print(dateparse.year)
print(dateparse.month)
print(dateparse.day)
print(dateparse.hour)

train.apply(lambda x: sum(x.isnull()) )

1、只有地理位置的数据有缺失，其他数据均无缺失。
2、总数据量是：23291027，缺失数据量为：15911010，7380017条数据是有地理位置信息的，占比为：31.68%
3、应该根据已有的地理位置的记录将缺少的地理位置都补全
    存在的情况：
        1.完全没有地理位置信息
        2.有地理位置信息，但是不一致，存在多个
            2.1、多个地理位置信息，明显很近
            2.2、多个地理位置信息，差距很大
        3.地理位置信息一致
        注：2,3描述的都是一个用户的行为记录中存在地理位置信息的情况即可（可能存在很多没有地理位置信息）
4、

#无意间发现一个思路，我可以先预测A会买or不会买，再进一步预测A会买什么!

#现在LR的思路是什么特征的用户会买，但这是通用的思考模式，是否能以推荐的模式来做这个预测，也即评价任意一个用户会不会买某一个商品的标准是个性化的而不是统一的！

#如果我无法找到个性化的预测的方式，那可以“排行榜--> 分类排行榜 --> 分人群个性化 --> 针对个人的个性化”的思路来，首先可以将人分为：

#果断型的、正常型、犹豫型的。根据直观的感觉，一般男性是果断型的，女性为犹豫比较型的。根据这两个维度可以求和交集，共分为6类！

#注：我确实没有男女的数据，但将用户行为分为两类，是不需要男女信息的。

#一个是点击率预估，一个是聚类。